In [2]:
!pip install -q requests beautifulsoup4 tqdm

You should consider upgrading via the 'C:\Users\Eunha\Documents\Personal work\ValorantAssistant\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
from bs4 import BeautifulSoup
import requests
import json
from tqdm import tqdm


## URLS

In [4]:
BASE_URL = "https://valorant.fandom.com"
AGENTS_PAGE = "https://valorant.fandom.com/wiki/Agents"

In [7]:
def get_agent_links():
    res = requests.get(AGENTS_PAGE)
    soup = BeautifulSoup(res.text, "html.parser")
    agent_links = []
    # Find the main agents table
    table = soup.find("table", class_="wikitable sortable")
    if not table:
        print("Could not find agents table")
        return []

    rows = table.find_all("tr")
    for row in rows:
        # Get the second cell which has agent link
        cells = row.find_all("td")
        if len(cells) > 1:
            cell = cells[1]
            link = cell.find("a")
            if link and link.get("href") and "/wiki/" in link.get("href"):
                agent_links.append(BASE_URL + link.get("href"))
    return list(set(agent_links))


def parse_agent_page(url):

    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")
    
    try:
        # Get basic info
        name = soup.find("h1", id="firstHeading").text.strip()


    
        info_box = soup.find_all('section', class_='pi-item pi-group pi-border-color pi-collapse pi-collapse-open')
        bio_box = info_box[0]
        game_details_box= info_box[1]
        bio_info = {}
        if bio_box:
            bio_field_mapping = {
                'realname': 'real_name',
                'aliases': 'aliases',
                'pronouns': 'pronouns',
                'origin': 'origin',
                'race': 'race',
                'number': 'agent_number',
            }

            # Find all data items in the info box
            for data_item in bio_box.find_all('div', class_='pi-item pi-data pi-item-spacing pi-border-color'):
                source = data_item.get('data-source')
                if source in bio_field_mapping:
                    value_div = data_item.find('div', class_='pi-data-value')
                    if value_div:
                        bio_info[bio_field_mapping[source]] = ' '.join(value_div.stripped_strings)

        #game details box
        game_info = {}
        if game_details_box:
                game_field_mapping = {
                'role': 'role',
                'basic-abilities': 'basic',
                'signature-abilities': 'signature',
                'ultimate-ability': 'ultimate',        


            }
            # for data_item in bio_box.find_all('div', class_='pi-item pi-data pi-item-spacing pi-border-color'):

        return {
            "agent": name,
            # "role": role,
            "biography": bio_info,  # Add the biographical information
            "abilities": game_info,
            "url": url
        }

    except Exception as e:
        print(f"Error parsing {url}: {e}")
        return None

In [8]:
def main():
    agent_links = get_agent_links()
    
    all_agents = []
 
    for link in tqdm(agent_links):
        agent_data = parse_agent_page(link)
        if agent_data:
            all_agents.append(agent_data)

    with open("data/valorant_agents.json", "w", encoding="utf-8") as f:
        json.dump(all_agents, f, indent=2, ensure_ascii=False)

    print(f"✅ Done! Scraped {len(all_agents)} agents.")

main()

100%|██████████| 27/27 [00:10<00:00,  2.56it/s]

✅ Done! Scraped 27 agents.
